### 문제 FewShotPromptTemplate과 시스템 메시지 활용 

뉴스 키워드 추출기

1) 문제 설명
FewShotPromptTemplate을 사용하여 뉴스 기사에서 핵심 키워드 3개를 추출하는 시스템을 구현해보세요. 주어진 예시들을 참고하여 일관된 형식으로 키워드를 추출해야 합니다.

2) 요구사항
FewShotPromptTemplate 사용
최소 3개의 예시(examples) 포함
뉴스 텍스트에서 핵심 키워드 3개 추출
일관된 출력 형식 유지
다양한 분야의 뉴스로 테스트


In [ ]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY)

In [6]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

# 1. 예시 프롬프트
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "{news}"),
    ("ai", "키워드: {keywords}")
])

# 2. 예시 데이터 3개 이상
examples = [
    {
        "news": "삼성전자가 내년 초에 자체적으로 개발한 인공지능(AI) 가속기를 처음으로 출시할 예정이다. 이는 AI 반도체 시장에서 지배적인 위치를 차지하고 있는 엔비디아의 독점을 도전하고, 세계 최고의 반도체 제조업체로서의 지위를 다시 확립하려는 삼성전자의 노력으로 해석된다.",
        "keywords": "삼성전자, 인공지능, 엔비디아"
    },
    {
        "news": "세계보건기구(WHO)는 최근 새로운 건강 위기에 대응하기 위해 국제 협력의 중요성을 강조했다. 전염병 대응 역량의 강화와 글로벌 보건 시스템의 개선이 필요하다고 발표했다.",
        "keywords": "세계보건기구, 건강위기, 국제협력"
    },
    {
        "news": "테슬라는 최근 완전자율주행(FSD) 소프트웨어의 업데이트 버전을 발표했다. 이번 업데이트에는 야간 운전 성능 향상과 새로운 센서 통합 기능이 포함되어 있다.",
        "keywords": "테슬라, 자율주행, 소프트웨어업데이트"
    }
]

# 3. Few-shot 프롬프트 구성
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples
)

# 4. 최종 프롬프트: 시스템 지시 + 예시 + 사용자 입력
final_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 뉴스 기사로부터 핵심 키워드 3개를 정확하고 간결하게 추출하는 전문가입니다."),
    few_shot_prompt,
    ("human", "{input}")
])

# 5. 모델 및 파서 설정
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.3
)
parser = StrOutputParser()

# 6. 체인 생성
chain = final_prompt | llm | parser

# 7. 테스트용 뉴스 기사 입력
test_news = (
    "제미나이 2.0 플래시는 현재 구글 AI 스튜디오(Google AI Studio) 및 버텍스 AI(Vertex AI)에서 제미나이 API를 통해 "
    "개발자에게 실험 모델로 제공됩니다. 모든 개발자는 멀티모달 입력 및 텍스트 출력을 사용할 수 있으며, "
    "텍스트 음성 변환(text-to-speech) 및 네이티브 이미지 생성은 일부 파트너들을 대상으로 제공됩니다. "
    "내년 1월에는 더 많은 모델 사이즈와 함께 일반에 공개될 예정입니다."
)

# 8. 실행
keywords_output = chain.invoke({"input": test_news})
print("📌 추출된 키워드:", keywords_output)


📌 추출된 키워드: 키워드: 제미나이, AI, 개발자
